[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/stfc/janus-core/blob/main/docs/source/tutorials/python/geom_opt.ipynb)

# Geometry Optimization

## Set up environment (optional)

These steps are required to run this tutorial with Google Colab. To do so, uncomment and run the cell below.

This will replace pre-installed versions of `numpy` and `torch` in Colab with versions that are known to be compatible with `janus-core`.

It may be possible to skip the steps that uninstall and reinstall `torch`, which will save a considerable amount of time.

These instructions but may work for other systems too, but it is typically preferable to prepare a virtual environment separately before running this notebook if possible.

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

# ! pip uninstall numpy -y # Uninstall pre-installed numpy

# ! pip uninstall torch torchaudio torchvision transformers -y # Uninstall pre-installed torch
# ! uv pip install torch==2.5.1 # Install pinned version of torch

# ! uv pip install janus-core[mace,sevennet,chgnet,visualise] data-tutorials --system # Install janus-core with MACE, SevenNet, CHGNet, and WeasWidget, and data-tutorials

# get_ipython().kernel.do_shutdown(restart=True) # Restart kernel to update libraries. This may warn that your session has crashed.

To ensure you have the latest version of `janus-core` installed, compare the output of the following cell to the latest version available at https://pypi.org/project/janus-core/

In [ ]:
from janus_core import __version__

print(__version__)

## Prepare data and modules

In [ ]:
from ase.io import read
from ase.optimize import FIRE
from weas_widget import WeasWidget

from data_tutorials.data import get_data

from janus_core.calculations.single_point import SinglePoint
from janus_core.calculations.geom_opt import GeomOpt

Use `data_tutorials` to get the data required for this tutorial:

In [ ]:
get_data(
    url="https://raw.githubusercontent.com/stfc/janus-tutorials/main/data/",
    filename=["NaCl-deformed.cif"],
    folder="data",
)

### Prepare for optimization of a deformed salt structure

In [ ]:
NaCl = read("data/NaCl-deformed.cif")
NaCl.wrap()
v=WeasWidget()
v.from_ase(NaCl)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

In [ ]:
sp_mace = SinglePoint(
    struct=NaCl.copy(),
    arch="mace_mp",
    device="cpu",
    model="small",
    calc_kwargs={"default_dtype": "float64"},
    properties="energy",
)

init_energy = sp_mace.run()["energy"]

To optimize only the atomic positions and not the cell, set `filter_class = None`:

In [ ]:
optimized_NaCl = GeomOpt(
    struct=sp_mace.struct,
    fmax=0.001,
    filter_class=None,
)

optimized_NaCl.run()
v=WeasWidget()
v.from_ase(optimized_NaCl.struct)
v.avr.model_style = 1
v.avr.show_hydrogen_bonds = True
v

Check energy has been lowered, and cell is unchanged:

In [ ]:
print(f"Initial cell: {NaCl.cell.cellpar()}")
print(f"Initial energy: {init_energy}")

print(f"Final cell: {optimized_NaCl.struct.cell.cellpar()}")
print(f"Final energy: {optimized_NaCl.struct.get_potential_energy()}")

### Optimizing cell vectors and atomic positions

Setting `filter_kwargs = {"hydrostatic_strain": True}` allows the cell lengths to be changed, in addition to atomic positions, but cell angles remain fixed:

In [ ]:
optimized_NaCl_lengths = GeomOpt(
    struct=NaCl.copy(),
    arch="mace_mp",
    device="cpu",
    model="small",
    calc_kwargs={"default_dtype": "float64"},
    fmax=0.001,
    filter_kwargs={"hydrostatic_strain": True},
)
optimized_NaCl_lengths.run()

Check energy has been lowered, and cell lengths have been updated, but angles remain unchanged:

In [ ]:
print(f"Initial cell: {NaCl.cell.cellpar()}")
print(f"Initial energy: {init_energy}")

print(f"Final cell: {optimized_NaCl_lengths.struct.cell.cellpar()}")
print(f"Final energy: {optimized_NaCl_lengths.struct.get_potential_energy()}")

### Optimizing at constant pressure and volume

Calculations can also be run at a fixed pressure and volume, by setting `filter_kwargs = {"scalar_pressure": x, "constant_volume": True}`

By default, both the cell lengths and angles will be optimized, in addition to the atomic positions.

We can also set the optimizer and filter class used, either by passing the function itself (e.g. `FIRE`) or passing the name of the ASE function (e.g. `"ExpCellFilter"`):

In [ ]:
optimized_NaCl_pressure = GeomOpt(
    struct=NaCl.copy(),
    arch="mace_mp",
    device="cpu",
    model="small",
    calc_kwargs={"default_dtype": "float64"},
    fmax=0.01,
    filter_kwargs={"scalar_pressure": 0.05, "constant_volume": True},
    optimizer=FIRE,
    filter_class="ExpCellFilter",
)
optimized_NaCl_pressure.run()

Check cell lengths and angles have both been updated:

In [ ]:
print(f"Initial cell: {NaCl.cell.cellpar()}")
print(f"Initial energy: {init_energy}")

print(f"Final cell: {optimized_NaCl_pressure.struct.cell.cellpar()}")
print(f"Final energy: {optimized_NaCl_pressure.struct.get_potential_energy()}")

## Comparing MACE to CHGNet and SevenNet

In [ ]:
optimized_NaCl_mace = GeomOpt(
    struct=NaCl.copy(),
    arch="mace_mp",
    device="cpu",
    model="small",
    calc_kwargs={"default_dtype": "float64"},
    fmax=0.01,
)
optimized_NaCl_mace.run()

optimized_NaCl_chgnet = GeomOpt(
    struct=NaCl.copy(),
    arch="chgnet",
    device="cpu",
    calc_kwargs={"default_dtype": "float64"},
    fmax=0.01,
)
optimized_NaCl_chgnet.run()

optimized_NaCl_sevennet = GeomOpt(
    struct=NaCl.copy(),
    arch="sevennet",
    device="cpu",
    calc_kwargs={"default_dtype": "float64"},
    fmax=0.01,
)
optimized_NaCl_sevennet.run()

In [ ]:
print(f"Initial energy: {init_energy}")

print(f"Final energy (MACE): {optimized_NaCl_mace.struct.get_potential_energy()}")
print(f"Final energy (CHGNet): {optimized_NaCl_chgnet.struct.get_potential_energy()}")
print(f"Final energy (SevenNet): {optimized_NaCl_sevennet.struct.get_potential_energy()}")